<a href="https://colab.research.google.com/github/yashtripya/ai_email_generator/blob/main/email_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

print("🔄 Loading the model. Please wait...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, trust_remote_code=True)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float32,
    device=0 if torch.cuda.is_available() else -1
)
print("✅ Model loaded successfully!")

def build_prompt(purpose, recipient, points, tone, sender):
    bullet_text = '\n'.join(f"- {point}" for point in points)
    return (
        f"Write a {tone} email to a {recipient} regarding: {purpose}.\n"
        f"Include the following key points:\n{bullet_text}\n"
        f"Start with a subject line, then write the body.\n"
        f"End the email with: \nRegards,\n{sender}\n\n"
        f"Subject:"
    )

def generate_text(prompt):
    output = generator(
        prompt,
        max_new_tokens=250,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )
    raw_text = output[0]["generated_text"].strip()
    return clean_output(raw_text, prompt)

def clean_output(text, prompt):
    if text.startswith(prompt):
        text = text[len(prompt):].strip()
    last_punc = max(text.rfind(p) for p in [".", "!", "?"])
    return text[:last_punc + 1].strip() if last_punc != -1 else text.strip()

def get_inputs():
    print("\n" + "=" * 60)
    print("📬 AI EMAIL GENERATOR".center(60))
    print("=" * 60)

    sender = input("✍️  Enter your name (Sender): ").strip()
    recipient = input("👤 Who is the recipient? (e.g., Manager, Friend): ").strip()
    purpose = input("📝 What is the purpose of the email? (e.g., job application, invite): ").strip()

    print("\n🧾 Enter bullet points you want included (type 'done' to finish):")
    points = []
    while True:
        item = input("➤ ").strip()
        if item.lower() == "done":
            break
        if item:
            points.append(item)

    print("\n🎯 Choose the tone of your email:\n1. Formal\n2. Informal")
    tone = "formal" if input("Select option (1 or 2): ").strip() == "1" else "informal"

    return purpose, recipient, points, tone, sender

def main():
    purpose, recipient, points, tone, sender = get_inputs()
    prompt = build_prompt(purpose, recipient, points, tone, sender)
    print("\n✨ Generating your email. Please wait...")
    result = generate_text(prompt)
    print("\n" + "=" * 60)
    print("📨 Generated Email:\n")
    print(result)

if __name__ == "__main__":
    main()
